In [1]:
from ultralytics import YOLO
import os

yolopath ="G:/ACI PROJECT/MODELS/YOLO/best.pt"
try:
    yolomodel = YOLO(yolopath)
    print("Model loaded successfully!")
except Exception as e:
    print(f"ERROR:{e}")

Model loaded successfully!


In [2]:
from transformers import AutoFeatureExtractor, ViTForImageClassification
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
deit_path = "G:\ACI PROJECT\MODELS\DEIT"
try:
    deit_model = ViTForImageClassification.from_pretrained(deit_path)
    deit_model.to(device).eval()
    emotion_feature_extractor = AutoFeatureExtractor.from_pretrained(deit_path, return_tensors = "pt")
    print("Model loaded successfully")
except Exception as e:
    print(f"ERROR:{e}")


c:\Users\User\miniconda3\envs\acienv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded successfully


c:\Users\User\miniconda3\envs\acienv\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
import cv2

cap = cv2.VideoCapture(0)

if cap.isOpened():
    print("camera is working properly")
else:    
    print("Could not open webcam")
    exit()
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("failed to capture image")
            break
        
        results = yolomodel(frame)
        #print(results)
        boxes = results[0].boxes.xywh.cpu().numpy()
        conf = results[0].boxes.conf.cpu().numpy()
        cls = results[0].boxes.cls.cpu().numpy() 
        labels = results[0].names
        for i, box in enumerate(boxes):
            x, y, w, h = box  # Unpack the bounding box coordinates
            label = labels[int(cls[i])]  # Get the label for the class
            confidence = conf[i]  # Confidence score

            # Crop detected face
            x1, y1 = max(int(x - w // 2), 0), max(int(y - h // 2), 0)
            x2, y2 = min(int(x + w // 2), frame.shape[1]), min(int(y + h // 2), frame.shape[0])
            cropped_face = frame[y1:y2, x1:x2]

            import torchvision.transforms as transforms
            transform = transforms.Compose(
                [
                    transforms.ToTensor()
                ]
            )

            cropped_face_tensor= transform(cropped_face).unsqueeze(0).to(device)

            if cropped_face.size > 0:
                deit_inputs = emotion_feature_extractor(images = cropped_face_tensor, return_tensors="pt").to(device)
            pixel_values = deit_inputs["pixel_values"].to(device)    
            with torch.no_grad():
                output = deit_model(pixel_values)
                emotion_class = torch.argmax(output.logits, dim=1).item()
                emotion_label = deit_model.config.id2label[emotion_class]
                

            # Draw bounding box and label
            cv2.rectangle(frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {confidence:.2f}, {emotion_label}", 
                (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        cv2.imshow("Webcam", frame)  

        if cv2.waitKey(1) & 0XFF == ord("q"):
            print("Exiting programme...")
            break
except KeyboardInterrupt:
    print("Program interrupted manually")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Windows closed...")


camera is working properly

0: 480x640 (no detections), 101.8ms
Speed: 5.5ms preprocess, 101.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 111.7ms
Speed: 2.0ms preprocess, 111.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 98.4ms
Speed: 2.1ms preprocess, 98.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 99.9ms
Speed: 4.4ms preprocess, 99.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 96.7ms
Speed: 2.0ms preprocess, 96.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 91.3ms
Speed: 4.3ms preprocess, 91.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 male, 94.8ms
Speed: 3.8ms preprocess, 94.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 male, 93.7ms
Speed: 4.8ms preproc